In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
catalog.list()

In [ ]:
articles = context.catalog.load('articles')

In [ ]:
article_descriptions = articles['detail_desc'].astype('str').to_list()

In [ ]:
len(article_descriptions)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
embeddings = model.encode(article_descriptions)

In [ ]:
embeddings[:5].shape

In [ ]:
text_embeddings = pd.DataFrame(data=embeddings, index=articles.article_id)

In [ ]:
text_embeddings.columns = [f'emb_{i+1}' for i, _ in enumerate(text_embeddings.columns)]

In [ ]:
text_embeddings

In [ ]:
text_embeddings.to_parquet('../data/04_feature/text_embeddings.pq')

In [ ]:
pd.read_parquet('../data/04_feature/text_embeddings.pq')

In [ ]:
pd.read_parquet('../data/04_feature/image_embeddings.pq')

### visualize embeddings

In [ ]:
from sklearn.neighbors import KDTree
from PIL import Image
import matplotlib.pyplot as plt
import os

In [ ]:
tree = KDTree(text_embeddings.values, leaf_size=5)

In [ ]:
def find_similar_images(query_article_id, embeddings, tree):
    _, ind = tree.query(embeddings.loc[query_article_id].values.reshape(1, -1), k=5)
    closest_embeddings = embeddings.iloc[ind[0]].index.tolist()
    imgs = [Image.open(f'../data/01_raw/images_128_128/{article_id}.jpg').convert('RGB') for article_id in closest_embeddings]
    _, axs = plt.subplots(1, 5, figsize=(12, 12))
    axs = axs.flatten()
    for img, article_id, ax in zip(imgs, closest_embeddings, axs):
        ax.title.set_text(f'{article_id}')
        ax.imshow(img)
    plt.show()

In [ ]:
find_similar_images('0680263001', text_embeddings, tree)

In [ ]:
def plot_n_random_similar_images(n, articles, embeddings):
    for i in range(n):
        random_article = articles.sample(1)
        print(f'''{i+1} random article:
        {random_article.prod_name},
        {random_article.article_id},
        {random_article.product_group_name}
        ''')
        find_similar_images(random_article.article_id, embeddings, tree)
        print('\n\n\n')

In [ ]:
plot_n_random_similar_images(5, articles, text_embeddings)

In [ ]:
for product_group in articles.product_group_name.unique():
    print(product_group)
    random_article = articles[articles['product_group_name']==product_group].sample(1)
    print(f'Random article id: {random_article.article_id}, random_article name: {random_article.prod_name}')
    find_similar_images(random_article.article_id, text_embeddings, tree)
    print('\n\n\n')

### tensorboard

In [ ]:
articles_set = set([file.stem for file in Path('../data/01_raw/images_128_128').glob('*.jpg')])

In [ ]:
embeddings_sample = text_embeddings.filter(items=articles_set, axis=0).sample(n=4_000)

In [ ]:
all_images = embeddings_sample.index.to_list()

In [ ]:
all_images[:5]

In [ ]:
embeddings_sample.to_csv('tensorboard_embeddings/feature_vecs.tsv', sep="\t", index=False, header=False)

In [ ]:
images_list = [Image.open(f'../data/01_raw/images_128_128/{article_id}.jpg').convert('RGB') for article_id in all_images]

In [ ]:
image_width, image_height = images_list[0].size

In [ ]:
one_square_size = int(np.ceil(np.sqrt(len(images_list))))

In [ ]:
master_width = image_width * one_square_size
master_height = image_height * one_square_size

In [ ]:
master_width, master_height

In [ ]:
spriteimage = Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent

In [ ]:
for count, image in enumerate(images_list):
    div, mod = divmod(count, one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod
    spriteimage.paste(image, (w_loc, h_loc))

In [ ]:
spriteimage.convert("RGB").save('tensorboard_embeddings/sprite.jpg', transparency=0)

In [ ]:
sample_articles = articles[articles.article_id.isin(all_images)].copy()

In [ ]:
sample_articles['article_id'] = pd.Categorical(sample_articles.article_id, categories=all_images, ordered=True)

In [ ]:
sample_articles.sort_values(by='article_id', inplace=True)

In [ ]:
sample_articles[['article_id', 'product_group_name', 'colour_group_name']].to_csv('tensorboard_embeddings/metadata.tsv', sep='\t', index=False)